In [ ]:
! pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp /content/drive/MyDrive/kaggle.json ~/
! mkdir ~/.kaggle
! mv ~/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d eldarsultanow/ml-bsd-dataset

ml-bsd-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip ml-bsd-dataset.zip

Archive:  ml-bsd-dataset.zip
replace rank-vs-ap.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
import numpy as np
import sympy
from scipy.interpolate import interp1d
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from sympy import prime, isprime, primerange
import random
import math
from sklearn.metrics import mean_absolute_error





In [ ]:
df = pd.read_csv("rank-vs-ap.csv", index_col=False)


In [ ]:
from sympy import primepi

def get_r_sequence(N:int):
  pi_N = primepi(N)
  res = [None]*pi_N

  for i in range(1,pi_N+1):
    res[i-1] = -1 + i*2/pi_N
  return res

print(len(get_r_sequence(1990)))
print(get_r_sequence(1990))


300
[-149/150, -74/75, -49/50, -73/75, -29/30, -24/25, -143/150, -71/75, -47/50, -14/15, -139/150, -23/25, -137/150, -68/75, -9/10, -67/75, -133/150, -22/25, -131/150, -13/15, -43/50, -64/75, -127/150, -21/25, -5/6, -62/75, -41/50, -61/75, -121/150, -4/5, -119/150, -59/75, -39/50, -58/75, -23/30, -19/25, -113/150, -56/75, -37/50, -11/15, -109/150, -18/25, -107/150, -53/75, -7/10, -52/75, -103/150, -17/25, -101/150, -2/3, -33/50, -49/75, -97/150, -16/25, -19/30, -47/75, -31/50, -46/75, -91/150, -3/5, -89/150, -44/75, -29/50, -43/75, -17/30, -14/25, -83/150, -41/75, -27/50, -8/15, -79/150, -13/25, -77/150, -38/75, -1/2, -37/75, -73/150, -12/25, -71/150, -7/15, -23/50, -34/75, -67/150, -11/25, -13/30, -32/75, -21/50, -31/75, -61/150, -2/5, -59/150, -29/75, -19/50, -28/75, -11/30, -9/25, -53/150, -26/75, -17/50, -1/3, -49/150, -8/25, -47/150, -23/75, -3/10, -22/75, -43/150, -7/25, -41/150, -4/15, -13/50, -19/75, -37/150, -6/25, -7/30, -17/75, -11/50, -16/75, -31/150, -1/5, -29/150, -14/75,

In [ ]:
primes = [-149/150, -74/75, -49/50, -73/75, -29/30, -24/25, -143/150, -71/75, -47/50, -14/15, -139/150, -23/25, -137/150, -68/75, -9/10, -67/75, -133/150, -22/25, -131/150, -13/15, -43/50, -64/75, -127/150, -21/25, -5/6, -62/75, -41/50, -61/75, -121/150, -4/5, -119/150, -59/75, -39/50, -58/75, -23/30, -19/25, -113/150, -56/75, -37/50, -11/15, -109/150, -18/25, -107/150, -53/75, -7/10, -52/75, -103/150, -17/25, -101/150, -2/3, -33/50, -49/75, -97/150, -16/25, -19/30, -47/75, -31/50, -46/75, -91/150, -3/5, -89/150, -44/75, -29/50, -43/75, -17/30, -14/25, -83/150, -41/75, -27/50, -8/15, -79/150, -13/25, -77/150, -38/75, -1/2, -37/75, -73/150, -12/25, -71/150, -7/15, -23/50, -34/75, -67/150, -11/25, -13/30, -32/75, -21/50, -31/75, -61/150, -2/5, -59/150, -29/75, -19/50, -28/75, -11/30, -9/25, -53/150, -26/75, -17/50, -1/3, -49/150, -8/25, -47/150, -23/75, -3/10, -22/75, -43/150, -7/25, -41/150, -4/15, -13/50, -19/75, -37/150, -6/25, -7/30, -17/75, -11/50, -16/75, -31/150, -1/5, -29/150, -14/75, -9/50, -13/75, -1/6, -4/25, -23/150, -11/75, -7/50, -2/15, -19/150, -3/25, -17/150, -8/75, -1/10, -7/75, -13/150, -2/25, -11/150, -1/15, -3/50, -4/75, -7/150, -1/25, -1/30, -2/75, -1/50, -1/75, -1/150, 0, 1/150, 1/75, 1/50, 2/75, 1/30, 1/25, 7/150, 4/75, 3/50, 1/15, 11/150, 2/25, 13/150, 7/75, 1/10, 8/75, 17/150, 3/25, 19/150, 2/15, 7/50, 11/75, 23/150, 4/25, 1/6, 13/75, 9/50, 14/75, 29/150, 1/5, 31/150, 16/75, 11/50, 17/75, 7/30, 6/25, 37/150, 19/75, 13/50, 4/15, 41/150, 7/25, 43/150, 22/75, 3/10, 23/75, 47/150, 8/25, 49/150, 1/3, 17/50, 26/75, 53/150, 9/25, 11/30, 28/75, 19/50, 29/75, 59/150, 2/5, 61/150, 31/75, 21/50, 32/75, 13/30, 11/25, 67/150, 34/75, 23/50, 7/15, 71/150, 12/25, 73/150, 37/75, 1/2, 38/75, 77/150, 13/25, 79/150, 8/15, 27/50, 41/75, 83/150, 14/25, 17/30, 43/75, 29/50, 44/75, 89/150, 3/5, 91/150, 46/75, 31/50, 47/75, 19/30, 16/25, 97/150, 49/75, 33/50, 2/3, 101/150, 17/25, 103/150, 52/75, 7/10, 53/75, 107/150, 18/25, 109/150, 11/15, 37/50, 56/75, 113/150, 19/25, 23/30, 58/75, 39/50, 59/75, 119/150, 4/5, 121/150, 61/75, 41/50, 62/75, 5/6, 21/25, 127/150, 64/75, 43/50, 13/15, 131/150, 22/25, 133/150, 67/75, 9/10, 68/75, 137/150, 23/25, 139/150, 14/15, 47/50, 71/75, 143/150, 24/25, 29/30, 73/75, 49/50, 74/75, 149/150, 1]


In [ ]:
abs_prime = [abs(prime) for prime in primes]

In [ ]:
ap_cols = [col for col in df.columns if col.startswith("ap_")]
df[ap_cols] = df[ap_cols].replace(0, np.nan)
df[ap_cols] = df[ap_cols].fillna(df[ap_cols].mean())

In [ ]:
for i in range(len(abs_prime)):
    ap_col = f"ap_{i+1}"
    df[ap_col].dropna(inplace=True)
    if (df[ap_col] == 0).any():
        df[ap_col].replace(0, df[ap_col].mean(), inplace=True)
    df[ap_col] = df[ap_col].apply(lambda x, p=abs_prime[i]: x / math.sqrt(p))



In [ ]:
modified_ap = pd.concat([df['rank'], df.iloc[:,1:301]], axis=1)

In [ ]:
print(modified_ap)

        rank      ap_1      ap_2      ap_3      ap_4      ap_5      ap_6  \
0          1  0.015302 -0.011886 -0.005834 -0.050121  0.065179 -0.027261   
1          1 -0.000975  0.276768 -0.005834 -0.025061 -0.001277  0.013630   
2          1  0.015302 -0.011886 -0.159145  0.050121 -0.043452 -0.040891   
3          0 -0.015302 -0.011886 -0.005834  0.125304 -0.010863  0.013630   
4          1 -0.015302 -0.276768  0.106097 -0.025061 -0.001277 -0.013630   
...      ...       ...       ...       ...       ...       ...       ...   
149994     3 -0.015302 -0.011886 -0.005834  0.050121  0.032589  0.040891   
149995     3  0.015302 -0.011886 -0.106097 -0.002706 -0.065179  0.034076   
149996     3 -0.000975 -0.138384 -0.159145 -0.025061  0.021726 -0.013630   
149997     3 -0.000975 -0.276768 -0.106097 -0.002706 -0.065179  0.027261   
149998     3 -0.000975 -0.276768 -0.212194  0.050121 -0.065179 -0.027261   

            ap_7      ap_8      ap_9  ...    ap_291    ap_292    ap_293  \
0      -0.00

In [ ]:
modified_ap.to_csv('final_ap.csv',index=False)

In [ ]:
df = pd.read_csv('final_ap.csv')

In [ ]:
subset = df.sample(frac=0.1, random_state=42)

In [ ]:
feature_col = [col for col in subset.columns if col.startswith('ap_')]
X = subset[feature_col]

In [ ]:
df_rank = df['rank'][:15000]

In [ ]:
y = df_rank
y = y.values

In [ ]:
X_train_reshaped = np.expand_dims(X_train, axis=-1)
X_test_reshaped = np.expand_dims(X_test, axis=-1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
sgd = SGD(lr=0.01, momentum=0.9)

model.compile(optimizer=sgd,
              loss='mse',
              metrics=['mae', 'mse',])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


In [ ]:
y_pred = model.predict(X_test)

abs_error = abs(y_test - y_pred)

mae = abs_error.mean()

accuracy = 1 - mae

print("Accuracy: ".format(accuracy))

94/94 [==============================] - 1s 12ms/step
Accuracy: 
